In [7]:
from langchain_community.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel

In [8]:
class PartyPlanningRetrieverTool(Tool):

    name = "party_planning_retriever"
    description = "Searches a knowledge base of party planning ideas. Use keywords like 'superhero', 'entertainment', 'catering', 'decoration', 'music', 'food' to find relevant ideas."
    inputs = {
        "query": {
            "type": "string",
            "description": "Search keywords (e.g., 'superhero entertainment', 'catering food', 'decoration ideas')"
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(docs, k = 5) #top 5 results

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Query must be a string."

        docs = self.retriever.invoke(query)

        return "\nRetrieved ideas:\n" + "".join(
            [f"\n\n==== Idea {str(i)} ====\n" + doc.page_content
             for i, doc in enumerate(docs)]
        )
    
# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

In [9]:
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

In [10]:
#Split the documents into smaller chuncks for better search results
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50,
                                               add_start_index=True,
                                               strip_whitespace=True,
                                               separators=["\n\n", "\n", " ", ""]
)

docs_processed = text_splitter.split_documents(source_docs)

In [ ]:
party_planning_retriever = PartyPlanningRetrieverTool(docs=docs_processed)

agent = CodeAgent(tools=[party_planning_retriever], model = InferenceClientModel(), verbosity_level=2)

response = agent.run("""
Search for party planning ideas about:
1. Entertainment options for a superhero party
2. Catering and food ideas 
3. Decoration suggestions

Use the party_planning_retriever tool to find relevant ideas.
""")

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for party planning ideas about:                                                                          │
│ 1. Entertainment options for a superhero party                                                                  │
│ 2. Catering and food ideas                                                                                      │
│ 3. Decoration suggestions                                                                                       │
│                                                                                                                 │
│ Use the party_planning_retriever tool to find relevant ideas.                                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will use the `party_planning_retriever` tool to gather ideas for entertainment, catering and food, and  
decoration suggestions for a superhero party.                                                                      
                                                                                                                   
<code>                                                                                                             
entertainment_ideas = party_planning_retriever(query="superhero entertainment")                                    
catering_food_ideas = party_planning_retriever(query="superhero catering food")                                    
decoration_ideas = party_planning_retriever(query="superhero decoration ideas")                                    
                                                                                                                   
print("Entertainment Ideas:\n", entertainment_ideas)                                                               
print("Catering and Food Ideas:\n", catering_food_ideas)                                                           
print("Decoration Ideas:\n", decoration_ideas)                                                                     
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  entertainment_ideas = party_planning_retriever(query="superhero entertainment")                                  
  catering_food_ideas = party_planning_retriever(query="superhero catering food")                                  
  decoration_ideas = party_planning_retriever(query="superhero decoration ideas")                                  
                                                                                                                   
  print("Entertainment Ideas:\n", entertainment_ideas)                                                             
  print("Catering and Food Ideas:\n", catering_food_ideas)                                                         
  print("Decoration Ideas:\n", decoration_ideas)                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Entertainment Ideas:
 
Retrieved ideas:


==== Idea 0 ====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

==== Idea 1 ====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

==== Idea 2 ====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

==== Idea 3 ====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

==== Idea 4 ====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.
Catering and Food Ideas:
 
Retrieved ideas:


==== Idea 0 ====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

==== Idea 1 ====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

==== Idea 2 ====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

==== Idea 3 ====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

==== Idea 4 ====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.
Decoration Ideas:
 
Retrieved ideas:


==== Idea 0 ====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

==== Idea 1 ====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

==== Idea 2 ====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

==== Idea 3 ====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

==== Idea 4 ====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

Out: None

...

Execution logs:
Final Entertainment Ideas:
 Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.
Final Catering and Food Ideas:
 'The Hulk's Green Smoothie'
'Iron Man's Power Steak'
Final Decoration Ideas:
 Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

Final answer: {'Entertainment Ideas': 'Interactive experiences with VR where guests can engage in superhero 
simulations or compete in themed games.', 'Catering and Food Ideas': "'The Hulk's Green Smoothie'\n'Iron Man's 
Power Steak'", 'Decoration Ideas': 'Decorate with iconic superhero logos and projections of Gotham and other 
superhero cities around the venue.\nA superhero-themed masquerade ball with luxury decor, including gold accents 
and velvet curtains.'}

[Step 7: Duration 16.27 seconds| Input tokens: 30,520 | Output tokens: 1,427]

{'Entertainment Ideas': 'Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.', 'Catering and Food Ideas': "'The Hulk's Green Smoothie'\n'Iron Man's Power Steak'", 'Decoration Ideas': 'Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.\nA superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.'}


: 